# 建立 Kd_Tree 

In [1]:
import numpy as np

# 分为两部分：Kd树的建立；Kd树的搜索

## 实现一个结点类

In [2]:
class Node: #结点
    def __init__(self, data, lchild = None, rchild = None):
        '''一个节点包括节点域，左子树，右子树'''
        self.data = data
        self.lchild = lchild
        self.rchild = rchild
 
class KdTree:   
    #kd树
    def __init__(self):
        self.kdTree = None
    
    def create(self, dataSet, depth):   
        '''创建kd树，返回根结点'''
        if (len(dataSet) > 0):
            m, n = np.shape(dataSet)    #求出样本行，列
            midIndex = int(m / 2) #中间数的索引位置
            axis = depth % n    #判断以哪个轴划分数据
            
            sortedDataSet = self.sort(dataSet, axis) #进行排序
            node = Node(sortedDataSet[midIndex]) #将节点数据域设置为中位数
            
            leftDataSet = sortedDataSet[: midIndex] #将中位数的左边创建2改副本
            rightDataSet = sortedDataSet[midIndex+1 :]
            print('左子树：',leftDataSet)
            print('右子树：',rightDataSet)
            
            node.lchild = self.create(leftDataSet, depth+1) #将中位数左边样本传入来递归创建树
            node.rchild = self.create(rightDataSet, depth+1)
            
            return node
        else:
            return None
    
    
    def sort(self, dataSet, axis):  
        '''采用冒泡排序，利用aixs作为轴进行划分'''
        sortDataSet = dataSet[:]    #由于不能破坏原样本，此处建立一个副本
        m, n = np.shape(sortDataSet)
        for i in range(m):
            for j in range(0, m - i - 1):
                if (sortDataSet[j][axis] > sortDataSet[j+1][axis]):
                    temp = sortDataSet[j]
                    sortDataSet[j] = sortDataSet[j+1]
                    sortDataSet[j+1] = temp
        print('冒泡排序',sortDataSet)
        return sortDataSet
    
    
    def preOrder(self, node):
        '''前序遍历'''
        if node != None:
            print('>>',node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)
 
    def search(self, tree, x):  
        ''' 
        (1)在kd树中找出包含目标点x的叶结点：从根结点出发，递归的向下访问kd树。若目标点当前维的坐标值小于切分点的坐标值，
        则移动到左子结点，否则移动到右子结点。
        直到子结点为叶结点为止；
        (2)以此叶结点为“当前最近点”；
        (3)递归的向上回退，在每个结点进行以下操作：
        (a） 如果该结点保存的实例点比当前最近点距目标点更近，则以该实例点为“当前最近点”；
        (b) 当前最近点一定存在于该结点一个子结点对应的区域。
        检查该子结点的父结点的另一个子结点对应的区域是否有更近的点。具体的，检查另一个子结点对应的区域是否与以目标点为球心、\
        以目标点与“当前最近点”间的距离为半径的超球体相交。如果相交，可能在另一个子结点对应的区域内存在距离目标更近的点，\
        移动到另一个子结点。接着，递归的进行最近邻搜索。如果不相交，向上回退。\
        （4） 当回退到根结点时，搜索结束。最后的“当前最近点”即为x的最近邻点。
        '''
        self.nearestPoint = None    #保存最近的点
        self.nearestValue = 0   #保存最近的值
        def travel(node, depth = 0):    #递归搜索
            if node != None:    #递归终止条件
                n = len(x)  #特征数
                axis = depth % n    #计算轴
                if x[axis] < node.data[axis]:   #如果数据小于结点，则往左结点找
                    travel(node.lchild, depth+1)
                else:
                    travel(node.rchild, depth+1)
 
                #以下是递归完毕后，往父结点方向回朔，对应算法3.3(3)
                distNodeAndX = self.dist(x, node.data)  #目标和节点的距离判断
                if (self.nearestPoint == None): #确定当前点，更新最近的点和最近的值，对应算法3.3(3)(a)
                    self.nearestPoint = node.data
                    self.nearestValue = distNodeAndX
                elif (self.nearestValue > distNodeAndX):
                    self.nearestPoint = node.data
                    self.nearestValue = distNodeAndX
 
                print(node.data, depth, self.nearestValue, node.data[axis], x[axis])
                if (abs(x[axis] - node.data[axis]) <= self.nearestValue):  #确定是否需要去子节点的区域去找（圆的判断），对应算法3.3(3)(b)
                    if x[axis] < node.data[axis]:
                        travel(node.rchild, depth+1)
                    else:
                        travel(node.lchild, depth + 1)
        travel(tree)
        return self.nearestPoint
    def dist(self, x1, x2): #欧式距离的计算
        return ((np.array(x1) - np.array(x2)) ** 2).sum() ** 0.5

In [3]:
if __name__ == '__main__':
    dataSet = [[2, 3],
               [5, 4],
               [9, 6],
               [4, 7],
               [8, 1],
               [7, 2]]
    x = [5, 3]
    kdtree = KdTree()
    tree = kdtree.create(dataSet, 0)
    print('前序遍历')
    kdtree.preOrder(tree)
    print(kdtree.search(tree, x))

冒泡排序 [[2, 3], [4, 7], [5, 4], [7, 2], [8, 1], [9, 6]]
左子树： [[2, 3], [4, 7], [5, 4]]
右子树： [[8, 1], [9, 6]]
冒泡排序 [[2, 3], [5, 4], [4, 7]]
左子树： [[2, 3]]
右子树： [[4, 7]]
冒泡排序 [[2, 3]]
左子树： []
右子树： []
冒泡排序 [[4, 7]]
左子树： []
右子树： []
冒泡排序 [[8, 1], [9, 6]]
左子树： [[8, 1]]
右子树： []
冒泡排序 [[8, 1]]
左子树： []
右子树： []
前序遍历
>> [7, 2]
>> [5, 4]
>> [2, 3]
>> [4, 7]
>> [9, 6]
>> [8, 1]
[2, 3] 2 3.0 2 5
[5, 4] 1 1.0 4 3
[4, 7] 2 1.0 4 5
[7, 2] 0 1.0 7 5
[5, 4]
